In [1]:
!pip install pint
!pip install fuzzywuzzy

import pandas as pd
import pint
from pint import UnitRegistry
import re
ureg = UnitRegistry()

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import notebook

     |████████████████████████████████| 194kB 9.0MB/s 


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [0]:
from fractions import Fraction

def fraction_to_decimal(string):
  return float(sum(Fraction(s) for s in string.split()))


In [0]:
I_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/INGR_FINAL_COMPL_MERGE.csv', index_col=0)

In [0]:
F_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/FoodData_Central/food.csv')

In [0]:
P_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/FoodData_Central/food_portion.csv')

In [0]:
I_F_P_DF = I_DF.merge(F_DF, on='fdc_id', how='left').merge(
    P_DF, on='fdc_id',how='left').drop(columns=[
                                                'food_category_id',
                                                'publication_date',
                                                'data_points',
                                                'footnote',
                                                'min_year_acquired'
                                                ])

## CASES:
Survey (FNDDS) Foods 
```
Household quantity :   portion_description 
Household unit.    :   portion_description
```
SR Legacy Foods
```
Household quantity :   amount 
Household unit.    :   modifier
```
Foundation Food
```
Household quantity :   amount 
Household unit.    :   measure_unit_id
```
Sub Sample Food:  
```
There is no data available
```




## Survey (FNDDS) Foods
```
Household quantity :   portion_description 
Household unit.    :   portion_description
```

In [24]:
SF_DF = I_F_P_DF[I_F_P_DF.data_type == 'survey_fndds_food']
SF_DF

,modifier_x,name,quantity,sub_qty,sub_unit,unit,recipe_index,count,fdc_id,data_type,description,id,seq_num,amount,measure_unit_id,portion_description,modifier_y,gram_weight
7,divided,white sugar,2.0,NaN,NaN,tablespoons,0,18279.0,343930.0,survey_fndds_food,"Sugar, white, granulated or lump",158656.0,1.0,NaN,9999.0,1 cup,10205,200.0
8,divided,white sugar,2.0,NaN,NaN,tablespoons,0,18279.0,343930.0,survey_fndds_food,"Sugar, white, granulated or lump",158657.0,2.0,NaN,9999.0,1 teaspoon,22000,4.2
9,divided,white sugar,2.0,NaN,NaN,tablespoons,0,18279.0,343930.0,survey_fndds_food,"Sugar, white, granulated or lump",158658.0,3.0,NaN,9999.0,1 individual packet,60876,3.5
10,divided,white sugar,2.0,NaN,NaN,tablespoons,0,18279.0,343930.0,survey_fndds_food,"Sugar, white, granulated or lump",158659.0,4.0,NaN,9999.0,Guideline amount per fl oz of beverage,63480,1.4
11,divided,white sugar,2.0,NaN,NaN,tablespoons,0,18279.0,343930.0,survey_fndds_food,"Sugar, white, granulated or lump",158660.0,5.0,NaN,9999.0,1 cube,60641,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2126564,melted,unsalted butter,2.0,NaN,NaN,tablespoons,68633,2030.0,343816.0,survey_fndds_food,"Butter, stick, unsalted",158074.0,7.0,NaN,9999.0,Guideline amount on regular sandwich,64546,14.0
2126565,melted,unsalted butter,2.0,NaN,NaN,tablespoons,68633,2030.0,343816.0,survey_fndds_food,"Butter, stick, unsalted",158075.0,8.0,NaN,9999.0,Guideline amount on large sandwich,64547,28.0
2126566,or as needed,vegetable oil,1.0,NaN,NaN,teaspoon,68633,6748.0,343866.0,survey_fndds_food,"Vegetable oil, NFS",158375.0,1.0,NaN,9999.0,1 cup,10205,218.0
2126567,or as needed,vegetable oil,1.0,NaN,NaN,teaspoon,68633,6748.0,343866.0,survey_fndds_food,"Vegetable oil, NFS",158376.0,2.0,NaN,9999.0,1 tablespoon,21000,13.6


### Parse household quantity and unit

In [8]:
SF_DF['household_qty'] = SF_DF.portion_description.str.split(" ",expand=True).iloc[:,0]
SF_DF['household_unit'] = ''

for index, row in notebook.tqdm(SF_DF.iterrows(), total=len(SF_DF)):
  try:
    if not isinstance(row['portion_description'], str):
      continue
    SF_DF.at[index, 'household_unit'] = row['portion_description'].replace(row['household_qty'], '')
  except:
    raise
    print("ERROR")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
SF_DF['household_unit'].fillna('',inplace=True)
SF_DF['unit'].fillna('',inplace=True)
SF_DF = SF_DF.drop_duplicates()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [0]:
SF_DF = SF_DF.fillna(value={'unit': '', 'quantity': '', 
                                'sub_unit': '', 'sub_qty': 0, 'sub_unit': '',
                                'household_unit': '','household_qty': 0})

In [11]:
def to_decimal(string):
  try:
    if re.match('.*-.*', string):
      left = fraction_to_decimal(string.split('-')[0])
      right = fraction_to_decimal(string.split('-')[1])
      return left + right
    return fraction_to_decimal(string)
  except:
    return 0

to_decimal('1-1/2')

1.5

In [0]:
SF_DF['quantity'] = SF_DF['quantity'].replace(to_replace=r'n', value='', regex=True)
SF_DF['quantity'] = SF_DF['quantity'].replace(to_replace=r'a', value='', regex=True)
SF_DF['quantity'] = SF_DF['quantity'].astype(str).apply(to_decimal).astype(float)
SF_DF['household_qty'] = SF_DF['household_qty'].astype(str).apply(to_decimal).astype(float)
SF_DF['ingr_mass'] = ''
SF_DF['sub_qty'] = SF_DF['sub_qty'].astype(str).apply(to_decimal).astype(float)
SF_DF['household_sub_unit'] = ''
SF_DF['household_sub_qty'] = ''
SF_DF['unit'] = SF_DF['unit'].astype(str)


In [0]:
SF_DF['household_unit'] = SF_DF['household_unit'].apply(lambda x: x.strip())

In [14]:
def parse_sf_modifier(df):
  for index, row in notebook.tqdm(df.iterrows(),total=len(df)):
    if not isinstance(row['household_unit'], float):
      try:

        match = re.findall('\((.*)\)', row.household_unit)[0]
        df.at[index, 'household_sub_qty'] = match.split()[0]
        df.at[index, 'household_sub_unit'] =  ' '.join(match.split()[1:])
        df.at[index, 'household_unit'] = row.household_unit.replace(match, '').replace('()', '')
      except:
        continue

parse_sf_modifier(SF_DF)


In [0]:
UNITS = {"cup": ["cups", "cup", "c.", "c"], 
         "gallon": ["gal", "gal.", "gallon", "gallons"], "ounce": ["oz", "oz.", "ounce", "ounces", "onz", "oza","fl. oz.", "fl oz", "fluid ounce", "fluid ounces""oz", "oz."],
         "pint": ["pt", "pt.", "pint", "pints"], "pound": ["lb", "lb.", "pound", "pounds"],
         "quart": ["qt", "qt.", "qts", "qts.", "quart", "quarts"],
         "tablespoon": ["tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs"],
         "teaspoon": ["tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons"],
         "gram": ["g", "g.", "gr", "gr.", "grm", "gram", "grams"], "kilogram": ["kg", "kg.", "kilogram", "kilograms"],
         "liter": ["l", "l.", "liter", "liters"], "milligram": ["mg", "mg.", "milligram", "milligrams"],
         "milliliter": ["ml", "ml.", "milliliter", "milliliters"]}

def unit_lookup(unit):
  """
  Check if ingredient unit and h_unit are in UNITS dictionary

  PARAMS:
    unit:   the unit of measurement to check

  RETURNS: 
    True/False:  if unit not in the dictionary
    unit:        the key of the unit if it was in the dictionary


  """
  match = False
  for key, values in UNITS.items():
    # If  unit exists in dictionary:
    if unit in values:
      match = True
      unit = key

  return match, unit

## Define Cases


1.   there is neither a qty nor unit for ingredient `('', '')('cooking spray')`
>SOLUTION: use serving size for branded food

2.   there is a qty but no unit for ingredient `('1', '') ('1', 'lemon')`
>SOLUTION: multiply ingredient quantity with serving quanity and multiply by serving size

3.   there is a unit but no qty for ingredient `('', 'cup')`  UNCOMMON

4.   there is a qty but no unit, AND a subunit for ingredient`('1', '', '(16 oz)')`
>SOLUTION: multiply ingredient qty by subunit and use that as the ingredient measure

5.   ingredient unit is of weight type and household unit is of volume type `('1', 'oz', '2', 'cup')`
>SOLUTION: directly convert from weight to serving size (if in grams)

6.   ingredient unit is volume type and household unit is weight type `('1', cup', '2', oz')`
>SOLUTION: get a general ballpark conversion from volume to weight and use that

7.   ingredient unit not a valid unit
>SOLUTION: use serving size of the food


In [0]:
def volume_to_grams(measure):
  """
  Just convert the volume to ml to get a ballpark estimate without knowing the density
  """
  return measure.to('ml').magnitude


def convert_to_grams(i_qty, i_unit, h_qty, h_unit, serving_size, serving_unit):
  """
  Convert to grams the measures that are included in the ingredients that were scraped for each recipe
  from allrecipes.com

  PARAMETERS:
  i_qty:        ingredient quantity in branded_joined table
  i_unit:       ingredient unit in branded_joined table
  h_qty:        household quantity in branded_joined table
  h_unit:       household unit in branded_joined table
  serving_size: serving size in branded_joined table
  serving_unit: serving unit in branded_joined table

  RETURNS: number of grams in the allrecipes ingredient
  """
  try:
    h_measure = h_qty * ureg(h_unit)
    i_measure = i_qty * ureg(i_unit)
    s_measure = serving_size * ureg(serving_unit)

    # CASE 5: ingredient unit is of mass type and household unit is of volume type ('1', 'oz', '2', 'cup')
    if i_measure.check('[mass]') and h_measure.check('[volume]'):
      # SOLUTION: directly convert from mass to serving size (if in grams)
      i_to_s = i_measure.to('grams')
      to_grams = i_to_s.magnitude / s_measure.magnitude
      grams = to_grams * serving_size

      return grams

    # CASE 6: ingredient unit is volume type and household unit is weight type ('1', cup', '2', oz')
    elif i_measure.check('[volume]') and h_measure.check('[mass]'):
    #SOLUTION: get a general ballpark conversion from volume to weight and use that
      # convert ingredient volume to mass
      return volume_to_grams(i_measure)

    # Normal case
    else:
      i_to_h = i_measure.to(h_measure)
      to_grams = i_to_h.magnitude / h_measure.magnitude
      grams = to_grams * serving_size
      return grams
  except Exception as e:
    print(e)
    return 100


In [0]:

def standardize_quantities(df):
  for index, row in notebook.tqdm(df.iterrows(), total=len(df)): 
    try: 
      h_unit = row['household_unit'].strip().lower() 
      i_unit = row['unit'].strip().lower()
      iMatch, i_unit = unit_lookup(i_unit)
      hMatch, h_unit = unit_lookup(h_unit)

      # Case 7: ingredient unit not a valid unit
      if not iMatch and not hMatch:
        # SOLUTION: just use serving size and multiply by ingredient quantity
        df.at[index, 'ingr_mass'] = row['gram_weight'] * row['quantity']
      elif iMatch and hMatch:
        df.at[index, 'ingr_mass'] = convert_to_grams(row['quantity'], i_unit, row['household_qty'], h_unit, row['gram_weight'],'g')
      
      # CASE 4: If there is a qty but no unit, AND a subunit for ingredient('1', '', '(16 oz)')
      elif (row['quantity'] != 0 and row['unit'] == '' and row['sub_qty'] != 0):
        # SOLUTION: multiply ingredient qty by subunit qty and use that as the ingredient measure
        qty = row['quantity'] * row['sub_qty']
        unit = row['sub_unit']
        df.at[index, 'ingr_mass'] = convert_to_grams(qty, unit, row['gram_weight'], 'gram', row['gram_weight'],'g')
      
      # CASE 1: If there is neither a qty nor unit for ingredient ('', '')('cooking spray')
      elif (row['quantity'] == 0 and i_unit == ''):
        # SOLUTION: Use serving size for branded food
        df.at[index, 'ingr_mass'] = row['gram_weight']

      # CASE 2: If there is a qty but no unit for ingredient ('1', '') ('1', 'lemon')
      elif (row['quantity'] != 0 and i_unit == ''):
        # Multiply ingredient quantity with serving quanity and multiply serving size
        df.at[index, 'ingr_mass'] = row['quantity'] * row['gram_weight']
      else:
        if(row['quantity'] != 0):
          df.at[index, 'ingr_mass'] = row['gram_weight'] * row['quantity']
        else:
          df.at[index, 'ingr_mass'] = row['gram_weight']
    except KeyboardInterrupt:
      continue
    # If error (all other cases):
    except Exception as e:
      # Just use serving size as total grams
      raise
      df.at[index, 'ingr_mass'] = row['gram_weight']
      continue

  return df

  


In [18]:
SF_DF = standardize_quantities(SF_DF)

'fluid' is not defined in the unit registry
'fluid' is not defined in the unit registry
'layer' is not defined in the unit registry
'layer' is not defined in the unit registry
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
'fluid' is not defined in the unit registry
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
Cannot convert from 'inch' ([length]) to 'gram' ([mass])
'fluid' is not defined in the unit registry
'fluid' is not defined in the unit registry
'fluid' is not defined in the unit registry
'fluid' is not define

In [0]:
SF_DF_clean = SF_DF.drop(columns=['data_type','id','seq_num','amount','measure_unit_id'])

In [0]:
SF_DF_clean.to_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/SURVEY_MASS.csv')

In [0]:
SF_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/SURVEY_MASS.csv')